In [63]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm

In [64]:
X_test = pd.read_csv('X_test_GgyECq8.csv')
X_train = pd.read_csv('X_train_Wwou3IE.csv')
y_train = pd.read_csv('y_train_jJtXgMX.csv')
y_pred = pd.read_csv('y_random_pt8afo8.csv')

In [65]:
X_train.columns

Index(['DELIVERY_START', 'load_forecast', 'coal_power_available',
       'gas_power_available', 'nucelear_power_available',
       'wind_power_forecasts_average', 'solar_power_forecasts_average',
       'wind_power_forecasts_std', 'solar_power_forecasts_std',
       'predicted_spot_price'],
      dtype='object')

In [66]:
y_train.columns

Index(['DELIVERY_START', 'spot_id_delta'], dtype='object')

In [67]:
def compute_weighted_accuracy(y_actual, y_pred):
    # If y_actual is a DataFrame, extract the 'spot_id_delta' column, otherwise assume it's already a numpy array
    if isinstance(y_actual, pd.DataFrame):
        actual = y_actual["spot_id_delta"].values
    else:
        actual = y_actual
    
    # If y_pred is a DataFrame, extract the 'spot_id_delta' column, otherwise assume it's already a numpy array
    if isinstance(y_pred, pd.DataFrame):
        predicted = y_pred["spot_id_delta"].values
    else:
        predicted = y_pred
    
    # actual = y_actual["spot_id_delta"].values
    # predicted = y_pred["spot_id_delta"].values

    correct_direction = (np.sign(actual) == np.sign(predicted)).astype(int)

    weights = np.abs(actual)

    weighted_accuracy = np.sum(correct_direction * weights) / np.sum(weights)

    return round(weighted_accuracy, 2)

In [68]:
def preprocess_data(df):
    # Ensure 'DELIVERY_START' is set as the index with timezone handling
    if 'DELIVERY_START' in df.columns:
        df['DELIVERY_START'] = pd.to_datetime(df['DELIVERY_START'], utc=True)  # Parse with timezone info
        df = df.set_index('DELIVERY_START')
        df.index = df.index.tz_convert('Europe/Berlin')  # Adjust to desired timezone, if needed

    # Add time-related features
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month

    # Handle missing values, encode categoricals, etc.
    df = df.fillna(0)  # Fill missing values with 0 (adjust as needed)
    df = pd.get_dummies(df, drop_first=True)  # Encode categorical variables

    # Drop unwanted columns
    df = df.drop(columns=['wind_power_forecasts_std', 'solar_power_forecasts_std'], errors='ignore')

    return df

def preprocess_y(y):
    # Ensure 'DELIVERY_START' is set as the index
    if 'DELIVERY_START' in y.columns:
        y['DELIVERY_START'] = pd.to_datetime(y['DELIVERY_START'], utc=True)  # Parse with timezone info
        y = y.set_index('DELIVERY_START')
        y.index = y.index.tz_convert('Europe/Berlin')  # Adjust to desired timezone
    return y

def split_train_data(X, y):
    # Ensure indices align during split
    split_size = len(X) // 3
    X_train1, X_train2, X_train3 = X.iloc[:split_size], X.iloc[split_size:2*split_size], X.iloc[2*split_size:]
    y_train1, y_train2, y_train3 = y.iloc[:split_size], y.iloc[split_size:2*split_size], y.iloc[2*split_size:]
    return X_train1, X_train2, X_train3, y_train1, y_train2, y_train3

def train_model(X_train, y_train):
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train.values.ravel())  # Ensure y is passed as a 1D array
    return model

def evaluate_model(model, X, y_actual):
    # Predict on the given dataset
    predictions = model.predict(X)
    
    # Compute regression metrics
    mse = mean_squared_error(y_actual, predictions)
    mae = mean_absolute_error(y_actual, predictions)
    
    # Use custom weighted accuracy
    y_pred_df = pd.DataFrame(predictions, index=X.index, columns=["spot_id_delta"])
    weighted_acc = compute_weighted_accuracy(y_actual, y_pred_df)
    
    return mse, mae, weighted_acc

def tune_hyperparameters(X_train2, y_train2):
    # Flatten y_train2 to 1D array
    y_train2 = y_train2.values.ravel()  # Ensure y is 1D array
    # Define the model
    rf_model = RandomForestRegressor(random_state=42)

    # Set up the hyperparameter grid
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 20],
    }

    # Perform GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
    
    grid_search.fit(X_train2, y_train2)

    # Return the best model and parameters
    return grid_search.best_estimator_, grid_search.best_params_

def launch(X_train, y_train, X_test):
    # Preprocess X_train and X_test
    X_train = preprocess_data(X_train)
    X_test = preprocess_data(X_test)

    # Preprocess y_train (only index adjustment)
    y_train = preprocess_y(y_train)

    # Split data into 3 parts
    X_train1, X_train2, X_train3, y_train1, y_train2, y_train3 = split_train_data(X_train, y_train)

    # Tune hyperparameters using the validation set (X_train2 and y_train2)
    best_model, best_params = tune_hyperparameters(X_train2, y_train2)

    # Train the model on the full training set (X_train1 + X_train2)
    X_train_full = pd.concat([X_train1, X_train2])
    y_train_full = pd.concat([y_train1, y_train2])

    # Flatten y_train_full to 1D array
    y_train_full = y_train_full.values.ravel()  # Ensure y is 1D array

    # Train the model on the full training data
    best_model.fit(X_train_full, y_train_full)

    # Evaluate the model on the full training data (X_train_full)
    train_mse, train_mae, train_weighted_acc = evaluate_model(best_model, X_train_full, y_train_full)

    # Validate on the validation set (X_train2, y_train2)
    val_mse, val_mae, val_weighted_acc = evaluate_model(best_model, X_train2, y_train2)

    # Test on the test set (X_train3, y_train3)
    test_mse, test_mae, test_weighted_acc = evaluate_model(best_model, X_train3, y_train3)

    # Apply model to the actual test dataset
    predictions_test = best_model.predict(X_test)
    y_pred_test = pd.DataFrame(predictions_test, index=X_test.index, columns=["spot_id_delta"])

    return best_model, (train_mse, train_mae, train_weighted_acc), (val_mse, val_mae, val_weighted_acc), (test_mse, test_mae, test_weighted_acc), y_pred_test

# Execute the pipeline
model, train_metrics, val_metrics, test_metrics, y_pred_test = launch(X_train, y_train, X_test)

# Print evaluation results
print("Best Model:", model)
print("Train Metrics - MSE:", train_metrics[0], "MAE:", train_metrics[1], "Weighted Accuracy:", train_metrics[2])
print("Validation Metrics - MSE:", val_metrics[0], "MAE:", val_metrics[1], "Weighted Accuracy:", val_metrics[2])
print('------------------------------------------------------------------------------------------------------')
print("Test Metrics - MSE:", test_metrics[0], "MAE:", test_metrics[1], "Weighted Accuracy:", test_metrics[2])

# Save predictions
y_pred_test.to_csv('y_pred_test.csv', index=False)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best Model: RandomForestRegressor(max_depth=5, n_estimators=200, random_state=42)
Train Metrics - MSE: 1236.642890502967 MAE: 19.883611270098992 Weighted Accuracy: 0.74
Validation Metrics - MSE: 1598.1035587384122 MAE: 26.34907172839559 Weighted Accuracy: 0.79
------------------------------------------------------------------------------------------------------
Test Metrics - MSE: 1388.2375254835695 MAE: 20.51665193117456 Weighted Accuracy: 0.42
